#Escuela Superior de Fisico y Matematicas

Licenciatura En Matematica Algoritmica

Algebra Abstracta

Identificacion de Irreducibles y generacion de Campos

Profesor: Jorge Aguilar Guzman

Alumno:Victor Hugo Martinez Huicochea

In [ ]:
import numpy
import sympy
from sympy.polys.domains import ZZ,GF
import copy
import itertools as it

#Fuciones de ayuda
Se utilizan las siguientes funciones:


*   **Inv** - Encuentra el inverso multiplicativo en $Z_p$ de un numero
*   **AlgDiv** - Hace algoritmo de la division a las funciones
*   **Irreducible** - Determina si un polinomio es reducible(False) o irreducible(True) mediante el estudio de sus raices ($Grado\leq3$) o por fuerza bruta (Grados superiores)

*   **Generador** - Genera todos los polinomios monicos posibles hasta el grado deseado menos 1 (Se utiliza para fuerza bruta)

*   **Monificador** - Devuelve el polinomio monico asociado al polinomio, junto con su escalar

*  **Campo**-  Genera el campo a trabajar


*  **CombLD** - Genera un conjunto de elementos linealmente dependiente a partir de un elemento del campo

*  **MinimoPolinomio** - Convierte los elementos de la combinacion lineal a una matriz para ser resuelta por Gauss-Jordan

*  **G_J** - Aplica Gauss Jordan a los elementos, y regresa una matriz de la combinacion lineal de los mismos

* **PolinomioMinimofuerzaBruta** - En caso de que falle Gauss.Jordan, se utiliza para calcular el polinomio minimo irreducible mendiante fuerza bruta
* **Generador2**- Genera todo el conjunto de prueba para fuerza bruta, ejecutandolo solo una vez, pero aumentando la eficienca

In [ ]:
def Inv(a,n):
  b=(a**(n-2))%n
  return b

def AlgDiv(f,g,n):
  #Obtenemos coeficientes
  Coef_Up=f.all_coeffs()
  Coef_Down=g.all_coeffs()
  #Obtenemos Grados
  GU=len(Coef_Up)
  GD=len(Coef_Down)
  #Obtenemos el inverso del primer elemento
  I=Inv(Coef_Down[0],n)



  #Guardaremos los coeficientes en este elemento
  Resul=[]
  for i in range(GU-(GD-1)):
    Resul.append(0)
  j=0
  #and (len(g.all_coeffs())!=1 or len(f.all_coeffs())!=1)
  while(len(f.all_coeffs())>=len(g.all_coeffs())):
    Coef=f.all_coeffs()
    #Igualamos al grado en el que deberiamos estar, esto en caso de que no exista
    while (len(Coef)!=GU-j):
      Coef.insert(0,0)



    #Creamos un Auxiliar para cada Iteracion
    Aux=[]
    #Dividimos el coeficiente de arriba por el de abajo para obtener el coeficiente del resultado, agregamos al resultado
    Aux.append((Coef[0]*I)%n)
    Resul[j]=Aux[0]

    #Lo convertimos a polinomio, esto igualando el array a el grado que deberia ser,  esto mendiante
    # X^n/X^m=X^(n-m), solo que le restamos la iteracion puesto que es el grado siguinte
    Grad=GU-GD-j
    for i in range(Grad):
      Aux.append(0)
    #Coenvertimos a polinomio
    S=sympy.Poly(Aux,x,domain=GF(n))
    #Se Multiplica el de abajo por el resultado de esta iteracion
    S=S*g
    #Se Resta el resultado para la nueva iteracion
    f=f-S
    #Obtenemos nueva iteracion

    j+=1
    if GD==1 or GU==1:
      break


  #Retornamos
  Resul=sympy.Poly(Resul,x,domain=GF(n))
  return Resul,f


def Irreducible(I,p):
  #Obtenemos el grado del polinomio
  Grado=len(I.all_coeffs())-1

  #Si es grado 3 o menor, entonces solo con las raices
  if Grado<=3 and Grado>1:
    for i in range(0,p):
      if I.subs(x,i)==0:
        return False
    return True
  #Caso lineal
  elif Grado==1:
    return True
  #Si no, fuerza bruta
  else:
    #generamos todos los posibles polinomios monicos que puedan dividir al elemento
    Todos=Generador(Grado,p)
    for i in Todos:
      #Convertimos a polinomio
      Aux=sympy.Poly(i,x,domain=GF(p))
      #Aplicamos algoritmo de la division
      a,r=AlgDiv(I,Aux,p)
      #Si resulta que el residuo es 0, entonces es reducible
      if r==0:
        return False
    return True


def Generador(Grado,p):
  #Guardamos todos los polinomios monicos posibles hasta el grado deseado
  Todos=[]

  #Creamos todas las combinatorias posibles hasta el grado+1
  #(Acabara en grado, en cada i genera todas las combinaciones
  #en un array de tamaño i Ej, si i=2 entonces [a,b] donde a y b son todas las posibles combinaciones de Zp)
  for i in range(1,int(Grado/2)+1):
    T=list(it.product(range(0,p),repeat=i))
    T=list(list(i) for i in T)

    #Agregamos que sea monico, es decir anexamos al inicio un 1, asi es monico, si es irreducible, todos sus escalares tambien lo son
    for j in T:
      j.insert(0,1)
      Todos.append(j)
  return Todos



#vuelve Monico un polinomio
def Monificador(P,p):
  #Obtenemos coeficientes
  I=P.all_coeffs()

  #Si es negativo
  if I[0]<0:
    #Encontramos a cual equivale en Zp
    I[0]=p+I[0]

  #Guardamos el escalar del monico para regresarlo
  Esc=sympy.Poly([I[0]],x,domain=GF(p))
  #Encontramos inverso
  Inverso=[Inv(I[0],p)]

  #Multiplicamos por el inverso para volverlo monico
  P=P*(sympy.Poly(Inverso,x,domain=GF(p)))

  return P,Esc


#genera el campo
def Campo(I,p):
  #Obtenemos el grado del polinomio
  Grado=len(I.all_coeffs())-1

  #Obtenemos todas las combinaciones posibles
  T=list(it.product(range(0,p),repeat=Grado))
  T=list(sympy.Poly(list(i),x,domain=GF(p)) for i in T)
  return T

def CombLD(I,Elem,p):
  #Obtenemos el grado del polinomio(Dim de F)
  Grado=len(I.all_coeffs())-1

  #Generamos a sus elementos que seran LD 1, a, a^2,...,a^(n-1)
  Generados=[sympy.Poly(1,x,domain=GF(p))]
  Aux=Generados[0]
  for i in range(Grado):
    _,Aux=AlgDiv(Aux*Elem,I,p)
    if Aux==1:
      break
    Generados.append(Aux)

  return Generados

def G_J(Mat,Grado,p,especial):
  #Aplicamops Gauss-Jordan, para ello comvertiremos a np.array
  Mat=numpy.array(Mat)
  shape=Mat.shape

  #Luego para el primer elemento en la diagonal
  for i in range(shape[1]-1):
    #Obtenemos su inverso
    Inver=Inv(Mat[i][i],p)
    #Multiplicamos por su inverso mod p para que la diagonal quede en 1
    Mat[i]=(Mat[i]*Inver)%p
    #Para todo elemento que no sea el renglon i
    for j in range(shape[0]):
      if i!=j:
        #Obtenemos cuantas veces debemos de restar 1 para que se haga 0
        Mod=Mat[j][i]
        #Restamos
        Mat[j]=(Mat[j]-Mat[i]*Mod)%p

  #Transponemos para obtener los resultados
  Mat=Mat.transpose()
  #Obtenemos los resultados
  Aux=Mat[Mat.shape[0]-1]

  #Obtenemos el coeficiente que sera el vector menos los resultados ya que es ld
  Sol=[]
  if especial==False:
    #volteamos para facilitar
    for i in Aux[::-1]:
      #anexamos los elementos
      Sol.append(-i%p)
  else:
    Aux=Aux[::-1]
    while len(Aux)!=shape[1]-1:
      Aux=numpy.delete(Aux,0)

    for i in Aux:
      Sol.append(-i%p)
  Sol.insert(0,1)
  return Sol

#Regresa una matriz para resolver por Gauss-Jordan, mediante el campo LD anteriormente generado
def MinimoPolinomio(Generados,p,Grado,Esp):
  Elem=[]
  #Para cada elemento de generados
  for i in Generados:
    #comvertimos a matriz
    Aux=i.all_coeffs()
    #Agregamos los 0 que faltan
    while len(Aux)!=Grado:
      Aux.insert(0,0)
    #Lo hacemos mod p (por si se sale)
    for j in range(len(Aux)):
      Aux[j]=Aux[j]%p
    #Agregamos el polinomio invertido, de modo que su menor grado sea el primero
    Elem.append(Aux[::-1])
  #transponemos para G-J
  Elem=numpy.array(Elem)
  Elem=Elem.transpose()
  if(Esp==False):
    #Evita errores o bugs al reacomodar filas
    for i in range(Grado):
      if Elem[i][i]==0:
        for j in range(i,Grado):
          if Elem[j][i]!=0:
            Elem[[i,j]]=Elem[[j,i]]
            break
  else:
    #Evita errores o bugs al reacomodar filas
    for i in range(len(Generados)):
      if Elem[i][i]==0:
        for j in range(i,Grado):
          if Elem[j][i]!=0:
            Elem[[i,j]]=Elem[[j,i]]
            break

  return Elem

In [ ]:
#G_J puede llegara fallar en casos muy concretos (Ej, sistemas no consistentes donde algunos elementos no aparezcan)
#Por ello se pueba mediante fuerza bruta para encontrar otra solucion pausible
def PolinomioMinimofuerzaBruta(I,T,Elem,p):


  #Encuentra la combinacion lineal
  for i in T:
    Aux=sympy.Poly(0,x,domain=GF(p))
    for j in range(0,len(i)):
      Aux=Aux+sympy.Poly(i[j],x,domain=GF(p))*(Elem**(len(i)-j-1))
    _,r=AlgDiv(Aux,I,p)

    if r==0:
      Prueba=sympy.Poly(i,x,domain=GF(p))
      print("Polinomio del cual es raiz: ",Prueba)
      Ton=Irreducible(Prueba,p)
      if Ton==True:
        return Prueba,Ton

  return Prueba,Ton

#Genera todo el conjunto de prueba para fuerza bruta del polinomio minimo raiz
def Generador2(Grado,p):
  #Guardamos todos los polinomios monicos posibles hasta el grado deseado
  Todos=[]

  #Creamos todas las combinatorias posibles hasta el grado+1
  #(Acabara en grado, en cada i genera todas las combinaciones
  #en un array de tamaño i Ej, si i=2 entonces [a,b] donde a y b son todas las posibles combinaciones de Zp)
  for i in range(1,Grado):
    T=list(it.product(range(0,p),repeat=i))
    T=list(list(i) for i in T)

    #Agregamos que sea monico, es decir anexamos al inicio un 1, asi es monico, si es irreducible, todos sus escalares tambien lo son
    for j in T:
      j.insert(0,1)
      Todos.append(j)
  return Todos

#Codigo Principal

In [ ]:
#Establecemos x como variable
x=sympy.symbols("x")


#Dimensionamos el campo Zp
print("Por favor digite el numero p (primo), para el Zp[x]")
p=int(input())
#Pedimos los polinomios
I = input("Favor de insertar su funcion:  \n IMPORTANTE: Cuando multiplique escriba el signo * (Ejemplo: xy = x*y)\n Cuando escriba una potencia escribir ** (Ejemplo: e^(xy)=e**(x*y)\n Funcion p(x):\n")

"""
Algunos Ejemplos Tratados
p=19
I=x**2+1
I=2*x**2+2
I=x**3+x**2+1
p=2
I=x**4+x**3+1

p=3
I=x**5+2*x**4+2*x**2+2

#Ejemplo de reducible
#p=5
I=4*x**2+2*x+4
I=x**2+2*x+1
"""




I = sympy.Poly(I,x,domain=GF(p))
print("\n\n\n\n\nPolinomio Dado:",I,"\n\n\n\n")

#Convertimos a monico para facilitar los calculos
Coef=I.all_coeffs()
#El escalar es por el que se multiplicara para obtener calculos
I,esc=Monificador(I,p)


#Checamos si es irreducible
Crit=Irreducible(I,p)

Ya_Creado=False

if Crit==False:
  print("Es reducible, por favor intente con otro polinomio...")
else:

  #si es irreducible
  #Obtenemos grado
  Grado=len(I.all_coeffs())-1


  #Genera campo
  T=Campo(I,p)
  chocolate=1

  #Para cada elemento del campo
  I=esc*I
  print("Es irreducible en Z_",p,"[X]. El campo Z_",p,"/I donde I=<",I,">, es:")

  for i in T:
    print("\n\n\nElemento",chocolate)
    #caso constante
    k=i.all_coeffs()
    if len(k)==1:
      print("El elemento ",i, "Es raiz del siguiente polinomio:   x+",(p-k[0])%p)

    #Caso no constante, resolvemos (Obtenemos sus generados y como no es LD entonces existe comb lineal, donde es raiz)
    else:
      print("El elemento ",i,"es raiz del siguiente polinomio:")
      #Obtenemos los generados
      Generados=CombLD(I,i,p)
      #print(minimop(I,Comb,i,p))
      #print(len(Generados),Grado)
      if len(Generados)!=Grado+1:
        #esp
        PR=sympy.Poly(x+1,x,domain=GF(p))
        especial=True
        Mat=MinimoPolinomio(Generados,p,Grado,especial)
        Pol=G_J(Mat,len(Generados)-1,p,especial)
        PR=sympy.Poly(Pol,x,domain=GF(p))
      else:
        especial=False
        Mat=MinimoPolinomio(Generados,p,Grado,especial)
        Pol=G_J(Mat,Grado,p,especial)
        PR=sympy.Poly(Pol,x,domain=GF(p))

      #Checamos si es irreducible
      Crit=Irreducible(PR,p)
      if Crit==False:

        if Ya_Creado==False:
          Ya_Creado=True
          Tod=Generador2(Grado+2,p)

        a,Ton=PolinomioMinimofuerzaBruta(I,Tod,i,p)
        if Ton==False:
          print("Es reducible, lo sentimos, el metodo actual no cubre este resultado, intentar con otro")
          break
        else:
          print("Es irreducible")

      else:
        print("Exponentes LD:",Generados)
        print("Polinomio del cual es Raiz:",PR)
        print("Es irreducible")

    chocolate+=1

Por favor digite el numero p (primo), para el Zp[x]
3
Favor de insertar su funcion:  
 IMPORTANTE: Cuando multiplique escriba el signo * (Ejemplo: xy = x*y)
 Cuando escriba una potencia escribir ** (Ejemplo: e^(xy)=e**(x*y)
 Funcion p(x):
x**5+2*x**4+2*x**2+2





Polinomio Dado: Poly(x**5 - x**4 - x**2 - 1, x, modulus=3) 




Es irreducible en Z_ 3 [X]. El campo Z_ 3 /I donde I=< Poly(x**5 - x**4 - x**2 - 1, x, modulus=3) >, es:



Elemento 1
El elemento  Poly(0, x, modulus=3) Es raiz del siguiente polinomio:   x+ 0



Elemento 2
El elemento  Poly(1, x, modulus=3) Es raiz del siguiente polinomio:   x+ 2



Elemento 3
El elemento  Poly(-1, x, modulus=3) Es raiz del siguiente polinomio:   x+ 1



Elemento 4
El elemento  Poly(x, x, modulus=3) es raiz del siguiente polinomio:
Exponentes LD: [Poly(1, x, modulus=3), Poly(x, x, modulus=3), Poly(x**2, x, modulus=3), Poly(x**3, x, modulus=3), Poly(x**4, x, modulus=3), Poly(x**4 + x**2 + 1, x, modulus=3)]
Polinomio del cual es Raiz: Poly(x**5 -